In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 3.9MB 51.7MB/s 
     |████████████████████████████████| 450kB 59.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=9ce607e1e4c8eb10613110946a5b7c6a2c391906a7e003f722d629722aa91cc5
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflo

In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
device_name = tf.test.gpu_device_name()
if device_name!='/device:GPU:0':
  raise SystemError('GPU deivce not found')
print('Found GPU at:{}'.format(device_name))

Found GPU at:/device:GPU:0


In [4]:
from google.colab import files
file = files.upload()

Saving x_train.pickle to x_train.pickle
Saving y_train.pickle to y_train.pickle


In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import pickle

def preprocess_input(x):
    # Change BRG => RGB
    x = x[:, :, :, ::-1]

    # Normalize
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

x_train = pickle.load(open("x_train.pickle","rb"))
y_train = pickle.load(open("y_train.pickle","rb"))

BATCH_SIZE = 50
EPOCHS = 100
preprocess_input(x_train)


# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 50 classes
predictions = Dense(50, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop',metrics=['accuracy'], loss='categorical_crossentropy')

# train the model on the new data for a few epochs
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")
model.fit_generator(aug.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) // BATCH_SIZE,
                    epochs=EPOCHS)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),metrics=['accuracy'], loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")
model.fit_generator(aug.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) // BATCH_SIZE,
                    epochs=EPOCHS)

Using TensorFlow backend.


87916544/87910968 [==============================] - 2s 0us/step
Epoch 1/100
1/1 [==============================] - 10s 10s/step - loss: 3.9847 - accuracy: 0.0400
Epoch 2/100
1/1 [==============================] - 0s 248ms/step - loss: 4.2714 - accuracy: 0.0600
Epoch 3/100
1/1 [==============================] - 1s 747ms/step - loss: 4.8352 - accuracy: 0.0800
Epoch 4/100
1/1 [==============================] - 1s 786ms/step - loss: 4.9850 - accuracy: 0.1000
Epoch 5/100
1/1 [==============================] - 1s 797ms/step - loss: 4.5945 - accuracy: 0.1000
Epoch 6/100
1/1 [==============================] - 1s 739ms/step - loss: 4.1860 - accuracy: 0.2200
Epoch 7/100
1/1 [==============================] - 1s 764ms/step - loss: 3.7674 - accuracy: 0.1600
Epoch 8/100
1/1 [==============================] - 1s 788ms/step - loss: 3.5560 - accuracy: 0.3000
Epoch 9/100
1/1 [==============================] - 1s 795ms/step - loss: 3.3893 - accuracy: 0.2400
Epoch 10/100
1/1 [===========================